# Worksheet 08

Name: Bargav Jagatha

UID: U80005052

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.6517306258853743, 5.516025245452042, 4.008384787827758, 4.23411694028567, 3.629984279341533, 5.455327214632914, 6.997556136918242, 5.251499336629243, 5.156428820428055, 5.064563289099195]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.015152616012864, 8.005013745820246, 7.340027499484982, 8.918221535960008, 8.705167879161756, 8.52788962937136, 6.513545950077837, 8.222202873157697, 7.160974002337633, 8.332715005252105]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    # print(coin_output)
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.332715005252105, 7.160974002337633, 5.064563289099195, 8.222202873157697, 5.156428820428055, 6.513545950077837, 8.52788962937136, 5.251499336629243, 6.997556136918242, 5.455327214632914]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

prior = 0.5, mean and variance

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [7]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_c = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.332715005252105, 7.160974002337633, 8.222202873157697, 8.52788962937136, 6.997556136918242]
[5.064563289099195, 5.156428820428055, 6.513545950077837, 5.251499336629243, 5.455327214632914]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.848267529407407,  mean_2 = 5.488272922173449
var_1 = 0.4064970925160448,  var_2 = 0.2795964586456331


/Users/bargav/anaconda3/envs/clean_ml/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [11]:
from scipy.stats import norm

prob_c0_x = [] # P(S_0 | X_i)
prob_c1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_c[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.332715005252105
probability of observing that point if it came from cluster 0 =  0.4824357884761442
probability of observing that point if it came from cluster 1 =  4.7877355935117623e-23
point =  7.160974002337633
probability of observing that point if it came from cluster 0 =  0.23501323464764745
probability of observing that point if it came from cluster 1 =  2.4125164352775454e-08
point =  5.064563289099195
probability of observing that point if it came from cluster 0 =  6.436077660442039e-11
probability of observing that point if it came from cluster 1 =  0.45257633945471093
point =  8.222202873157697
probability of observing that point if it came from cluster 0 =  0.6428363982542802
probability of observing that point if it came from cluster 1 =  2.469123209408658e-21
point =  5.156428820428055
probability of observing that point if it came from cluster 0 =  2.948820193931935e-10
probability of observing that point if it came from cluster 1 =  0.7054929683489384
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [12]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]

mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]

var = [sum([x[0] * ((x[1] - mean[0]) ** 2) for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * ((x[1] - mean[1]) ** 2) for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5722820917134529,  P(S_2) = 0.4277179082865472
mean_1 = 7.679684982725497,  mean_2 = 5.31500785011061
var_1 = 0.5517456215633024,  var_2 = 0.11918307830231394


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [13]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        # print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        # prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


8.332715005252105
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 4.269889455160331e-139

7.160974002337633
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 4.3531971812219014e-52

5.064563289099195
Probability of coming from C_1 = 3.479765188604179e-05
Probability of coming from C_2 = 0.999965202348114

8.222202873157697
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 3.5136786997528865e-129

5.156428820428055
Probability of coming from C_1 = 2.0130300201167644e-05
Probability of coming from C_2 = 0.9999798696997989

6.513545950077837
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 3.5421110090524815e-21

8.52788962937136
Probability of coming from C_1 = 1.0
Probability of coming from C_2 = 1.7762151586004168e-157

5.251499336629243
Probability of coming from C_1 = 2.0742347161171083e-05
Probability of coming from C_2 = 0.9999792576528388

6.997556136918242
Probability of coming from C_1 = 1.0
Proba

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [14]:
probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])

    if p[1] > p[2]:
        print("belongs to cluster 1\n")
    elif p[1] < p[2]:
        print("belongs to cluster 2\n")
    else:
        print("Ambiguous\n")
    


8.332715005252105
belongs to cluster 1

7.160974002337633
belongs to cluster 1

5.064563289099195
belongs to cluster 2

8.222202873157697
belongs to cluster 1

5.156428820428055
belongs to cluster 2

6.513545950077837
belongs to cluster 1

8.52788962937136
belongs to cluster 1

5.251499336629243
belongs to cluster 2

6.997556136918242
belongs to cluster 1

5.455327214632914
belongs to cluster 2

